# Keypoint Detection

## Importe

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from matplotlib.image import imread
%matplotlib inline

## Pfade festlegen

In [3]:
df_train=pd.read_csv('train.csv', sep=';')
df_test=pd.read_csv('test.csv', sep=';')
train_path = 'Training'+os.sep
train_klein_path = 'TrainingKleineBilder'+os.sep
test_path = 'Test'+os.sep

## Bilder skalieren 

In [4]:
from PIL import Image

for filename in os.listdir(train_klein_path):
    os.remove(train_klein_path+filename)

#basewidth:
basewidth = 320
wpercent = (basewidth / float(1000))
hsize = int((float(1000) * float(wpercent)))

cropbox = (0,10,1000,1000)

for image_filename in (os.listdir(train_path)):
    if image_filename=='.DS_Store':
        continue
    image = Image.open(train_path+image_filename)
    new_image = image.crop(cropbox)
    new_image = new_image.resize((basewidth, hsize))
    new_image.save(train_klein_path+image_filename)

## Bilder in Array einlesen 

In [5]:
import re
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

dirlist = sorted_alphanumeric(os.listdir(train_klein_path))

images_train = []
for image_filename in dirlist:
    if image_filename=='.DS_Store':
        continue
    img = imread(train_klein_path+image_filename)
    images_train.append(img)

## Koordinaten skalieren und über Bilder legen 

### Koordinaten skalieren 

In [6]:
df_train = df_train * float(wpercent)

### Punkt in Bild einzeichnen 

In [7]:
def plot_image(df_images, df, index):
    image = plt.imshow(df_images[index])
    l = []
    l.append(plt.plot(df.loc[index][0], df.loc[index][1], 'ro'))
    return image, l

In [12]:
fig = plt.figure(figsize=(20, 20))

for i in range(20):
    ax = fig.add_subplot(5, 4, i+1)
    plot_image(images_train, df_train, i)
plt.show()

SyntaxError: unexpected EOF while parsing (<ipython-input-12-3e7ab8d07610>, line 6)

## Bilder und Koordinaten in np Arrays umwandeln 

In [9]:
images_train = np.array(images_train,dtype = 'float')
images_train = np.asarray(images_train, dtype=np.uint8).reshape(images_train.shape[0],basewidth,basewidth,1)

In [10]:
y = []
for i in range(0,df_train.shape[0]):
    point = df_train.iloc[i,:]
    y.append(point)
    
y = np.array(y, dtype = 'float')

## Daten in Trainings- , Test- und Validierungsdaten aufteilen 

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_rest, y_train, y_rest = train_test_split(images_train, y, test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_rest, y_rest, test_size=0.5, random_state=42)

## Modell 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, AvgPool2D, BatchNormalization, Dropout, Activation, MaxPooling2D
from keras import regularizers
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D

In [ ]:
model = Sequential()

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False, input_shape=(basewidth,basewidth,1)))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(2))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [ ]:
history=model.fit(x_train, y_train, epochs = 500, batch_size = 64, validation_data = (x_val, y_val))

In [ ]:
model.save("KeypointDetection.h5")

## Vorhersage des Models 

In [ ]:
evaluate = model.evaluate(x_test, y_test)
evaluate

In [ ]:
prediction = model.predict(x_test)

In [ ]:
prediction[1]

## Vorhergesagte Punkte in Bild einzeichnen 

In [ ]:
def plot_test_image(df,prediction, index):
    image = plt.imshow(df[index])
    l = []
    l.append(plt.plot(prediction[index][0], prediction[index][1], 'ro'))
        
    return image, l

In [ ]:
fig = plt.figure(figsize=(20, 20))

for i in range(5):
    ax = fig.add_subplot(5, 4, i + 1)
    plot_test_image(x_test,prediction, i)

plt.show()